In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import dotenv_values
import os
import googlemaps
from datetime import datetime
from tqdm import tqdm
import re

config = dotenv_values('../../.env')

DATA_PATH = Path('../../data')

df = pd.read_csv(DATA_PATH.joinpath('raw', 'concat_data.csv'))

df['Address'] = df['Address'].apply(lambda a: str(a).lower() + ', boston, ma')

gmaps = googlemaps.Client(key=config['MAPS_API_KEY'])

# Geocoding all addresses
lat_long = {'lat': [], 'lng': []}
for idx, row in tqdm(df.iterrows()):
    geocode_result = gmaps.geocode(row['Address'])
    res = geocode_result[0]["geometry"]["location"]
    lat_long['lat'] = res['lat']
    lat_long['lng'] = res['lng']
    
lat_long = pd.DataFrame(lat_long, index=np.arange(df.shape[0]))
df = pd.concat([df, lat_long], axis=1)

df.to_csv(DATA_PATH.joinpath('raw', 'lat_long.csv'), index=False)